In [13]:
import pandas as pd, numpy as np, plotly.express as px, ipywidgets as widgets
from ipywidgets import fixed


def get_data(path):
    read_path = pd.read_csv(path)
    read_path = read_path.drop(columns=['sqft_above', 'sqft_basement', 'sqft_living15', 'sqft_lot15', 'yr_renovated'])
    read_path['date'] = pd.to_datetime(read_path['date']).dt.strftime('%d-%m-%Y')
    read_path['year'] = np.int64(pd.to_datetime(read_path['date']).dt.strftime('%Y'))
    read_path['age'] = read_path['year'] - read_path['yr_built']
    
    read_path['bathrooms'] = np.int64(round(read_path['bathrooms'] - 0.3))
    read_path['floors'] = np.int64(round(read_path['floors'] - 0.3))
    read_path['M2_living'] = round(read_path['sqft_living'] / 10.764, 2)
    read_path = read_path.drop(columns=['sqft_living'])
    read_path['level'] = read_path['price'].apply(
                                    lambda x: 0 if (x >= 0) & (x < 321950) else
                                    1 if (x >=321950) & (x < 450000) else
                                    2 if (x >= 450000) & (x < 645000) else
                                    3
                                    )

    read_path['waterfront'] = read_path['waterfront'].apply(lambda x: 'no' if x == 0 else
                                             'yes'
                                             )
    list_index = read_path[read_path['age'] < 0].index
    read_path = read_path.drop(list_index, axis=0)

    return read_path


data = get_data('Datasets/kc_house_data.csv')
data.head()

,id,date,price,bedrooms,bathrooms,sqft_lot,floors,waterfront,view,condition,grade,yr_built,zipcode,lat,long,year,age,M2_living,level
0,7129300520,13-10-2014,221900.0,3,1,5650,1,no,0,3,7,1955,98178,47.5112,-122.257,2014,59,109.62,0
1,6414100192,09-12-2014,538000.0,3,2,7242,2,no,0,3,7,1951,98125,47.7210,-122.319,2014,63,238.76,2
2,5631500400,25-02-2015,180000.0,2,1,10000,1,no,0,3,6,1933,98028,47.7379,-122.233,2015,82,71.53,0
3,2487200875,09-12-2014,604000.0,4,3,5000,1,no,0,5,7,1965,98136,47.5208,-122.393,2014,49,182.09,2
4,1954400510,18-02-2015,510000.0,3,2,8080,1,no,0,3,8,1987,98074,47.6168,-122.045,2015,28,156.08,2


In [14]:

style = {'description_width': 'initial'}

price_limit = widgets.IntSlider(
                                value=data['price'].mean(),
                                min=data['price'].min(),
                                max=data['price'].max(),
                                step=1,
                                description='price',
                                disable=False,
                                style=style
)
waterfront_bar = widgets.Dropdown(options=data['waterfront'].unique().tolist(),
                                 value='no',
                                 disabled=False,
                                 description='waterfront')

def enter_widget(df, waterfront, price):
    houses = df[(df['price']<=price) & (df['waterfront'] == waterfront)][['id', 'waterfront', 'price', 'lat', 'long', 'level']]
    
    mapa = px.scatter_mapbox(houses, 
                        lat='lat',
                        lon='long',
                        color='level',
                        size='price',
                         mapbox_style='open-street-map',
                         zoom=10
                        )
    mapa.show()

widgets.interactive(enter_widget, df=fixed(data), waterfront=waterfront_bar, price=price_limit)

interactive(children=(Dropdown(description='waterfront', options=('no', 'yes'), value='no'), IntSlider(value=5…